# 🤟 ASL Alphabet Recognition - Model Training

Training CNN model untuk mengenali 25 huruf ASL (A-Z tanpa J)

**Dataset:** Kaggle ASL Alphabet  
**Input:** 28x28 grayscale images  
**Output:** 25 classes (A-Z without J)  
**Goal:** Model yang robust untuk webcam real-world

## 📦 Setup & Dependencies

In [ ]:
# Install dependencies
!pip install -q tensorflow pillow matplotlib scikit-learn opencv-python

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
import zipfile
from google.colab import files

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## 📥 Download Dataset

**Option 1:** Download dari Kaggle (Recommended)

1. Buat Kaggle API token:
   - Go to https://www.kaggle.com/settings
   - Create New API Token
   - Upload `kaggle.json` di cell berikutnya

**Option 2:** Upload dataset manual
- Download: https://www.kaggle.com/grassknoted/asl-alphabet
- Upload zip file

In [ ]:
# OPTION 1: Kaggle API (RECOMMENDED)

# Upload kaggle.json
print("Upload your kaggle.json file...")
uploaded = files.upload()

# Setup Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download dataset
!kaggle datasets download -d grassknoted/asl-alphabet

# Extract
!unzip -q asl-alphabet.zip -d asl_data
print("✓ Dataset downloaded and extracted!")

In [ ]:
# OPTION 2: Manual Upload (jika Option 1 tidak bisa)

# Uncomment jika pakai manual upload:
# print("Upload asl-alphabet.zip file...")
# uploaded = files.upload()
# !unzip -q asl-alphabet.zip -d asl_data
# print("✓ Dataset extracted!")

## 📂 Explore Dataset

In [ ]:
# Check dataset structure
data_dir = 'asl_data/asl_alphabet_train/asl_alphabet_train'

# List all classes
classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
print(f"Total classes: {len(classes)}")
print(f"Classes: {classes}")

# Remove 'J' and other non-letter classes
valid_letters = [c for c in classes if c in 'ABCDEFGHIKLMNOPQRSTUVWXYZ']  # No J
print(f"\nValid ASL letters (without J): {len(valid_letters)}")
print(f"Letters: {valid_letters}")

# Count images per class
for letter in valid_letters[:5]:  # Show first 5
    letter_dir = os.path.join(data_dir, letter)
    num_images = len(os.listdir(letter_dir))
    print(f"{letter}: {num_images} images")

In [ ]:
# Visualize sample images
def show_samples(num_samples=5):
    fig, axes = plt.subplots(5, 5, figsize=(15, 15))
    fig.suptitle('Sample ASL Letters', fontsize=16)
    
    for idx, letter in enumerate(valid_letters[:25]):
        letter_dir = os.path.join(data_dir, letter)
        images = os.listdir(letter_dir)[:1]  # Take first image
        
        img_path = os.path.join(letter_dir, images[0])
        img = Image.open(img_path)
        
        row = idx // 5
        col = idx % 5
        axes[row, col].imshow(img)
        axes[row, col].set_title(f"{letter}", fontsize=14, fontweight='bold')
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

show_samples()

## 🔄 Data Preprocessing with Hand Cropping

In [ ]:
# Load and preprocess data
IMG_SIZE = 28
LABELS = valid_letters
NUM_CLASSES = len(LABELS)

print(f"Image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Labels: {LABELS}")

In [ ]:
def crop_hand_region(image):
    """
    Crop image to hand region only (remove background)
    Using simple thresholding and bounding box detection
    """
    # Convert to grayscale
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    
    # Threshold to separate hand from background
    # Assuming hand is brighter than background
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return image  # Return original if no contours found
    
    # Get largest contour (should be hand)
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    
    # Add margin (10%)
    margin = int(max(w, h) * 0.1)
    x = max(0, x - margin)
    y = max(0, y - margin)
    w = min(image.width - x, w + 2*margin)
    h = min(image.height - y, h + 2*margin)
    
    # Crop
    cropped = image.crop((x, y, x+w, y+h))
    return cropped

def preprocess_image(img_path, use_crop=True):
    """
    Load and preprocess image:
    1. Load image
    2. Crop hand region (optional)
    3. Convert to grayscale
    4. Resize to 28x28
    5. Normalize to [0, 1]
    """
    # Load image
    img = Image.open(img_path)
    
    # Crop hand region
    if use_crop:
        try:
            img = crop_hand_region(img)
        except:
            pass  # Use original if cropping fails
    
    # Convert to grayscale
    img = img.convert('L')
    
    # Resize
    img = img.resize((IMG_SIZE, IMG_SIZE), Image.Resampling.LANCZOS)
    
    # Convert to array and normalize
    img_array = np.array(img, dtype=np.float32) / 255.0
    
    return img_array

# Test preprocessing
test_letter = LABELS[0]
test_dir = os.path.join(data_dir, test_letter)
test_img = os.path.join(test_dir, os.listdir(test_dir)[0])

# Original vs Cropped
img_original = preprocess_image(test_img, use_crop=False)
img_cropped = preprocess_image(test_img, use_crop=True)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(img_original, cmap='gray')
axes[0].set_title('Original (No Crop)')
axes[0].axis('off')
axes[1].imshow(img_cropped, cmap='gray')
axes[1].set_title('Hand Cropped')
axes[1].axis('off')
plt.show()

print("✓ Preprocessing test completed!")

## 📊 Load Dataset

In [ ]:
# Load all images (this may take a few minutes)
MAX_IMAGES_PER_CLASS = 3000  # Limit untuk speed (total 75,000 images)
USE_HAND_CROP = True  # Set True untuk hand cropping

X = []
y = []

print("Loading images...")
for label_idx, letter in enumerate(LABELS):
    letter_dir = os.path.join(data_dir, letter)
    images = os.listdir(letter_dir)[:MAX_IMAGES_PER_CLASS]
    
    print(f"Loading {letter}: {len(images)} images...", end=" ")
    
    for img_name in images:
        img_path = os.path.join(letter_dir, img_name)
        try:
            img_array = preprocess_image(img_path, use_crop=USE_HAND_CROP)
            X.append(img_array)
            y.append(label_idx)
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            continue
    
    print("✓")

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Reshape untuk Keras (add channel dimension)
X = X.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

print(f"\n✓ Dataset loaded!")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"Total images: {len(X)}")
print(f"Images per class: ~{len(X) // NUM_CLASSES}")

## ✂️ Train/Validation Split

In [ ]:
# Split data
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  # Ensure balanced split
)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"\nClass distribution (training):")
unique, counts = np.unique(y_train, return_counts=True)
for label_idx, count in zip(unique[:5], counts[:5]):
    print(f"  {LABELS[label_idx]}: {count} images")
print(f"  ...")

## 🧠 Build Model

In [ ]:
def create_model():
    """
    Create CNN model with:
    - 3 Convolutional blocks
    - Batch Normalization
    - Dropout for regularization
    - Dense layers
    """
    model = keras.Sequential([
        # Input layer
        layers.Input(shape=(IMG_SIZE, IMG_SIZE, 1)),
        
        # Conv Block 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Conv Block 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Conv Block 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.4),
        
        # Dense layers
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        
        # Output layer
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    return model

# Create model
model = create_model()

# Compile
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Summary
model.summary()

## 🎓 Data Augmentation

In [ ]:
# Data augmentation untuk meningkatkan robustness
data_augmentation = keras.Sequential([
    layers.RandomRotation(0.1),  # ±10 degrees
    layers.RandomZoom(0.1),  # ±10% zoom
    layers.RandomTranslation(0.1, 0.1),  # ±10% shift
])

# Visualize augmentation
sample_img = X_train[0:1]
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
fig.suptitle('Data Augmentation Examples', fontsize=14)

for i in range(8):
    augmented = data_augmentation(sample_img, training=True)
    row = i // 4
    col = i % 4
    axes[row, col].imshow(augmented[0].numpy().squeeze(), cmap='gray')
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

print("✓ Augmentation configured")

## 🚀 Training

In [ ]:
# Callbacks
callbacks = [
    # Early stopping
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Learning rate reduction
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    
    # Model checkpoint
    keras.callbacks.ModelCheckpoint(
        'best_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Training parameters
EPOCHS = 50
BATCH_SIZE = 64

print(f"Starting training...")
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print("\nTraining started...\n")

# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

print("\n✓ Training completed!")

## 📈 Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], label='Training')
axes[0].plot(history.history['val_accuracy'], label='Validation')
axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(history.history['loss'], label='Training')
axes[1].plot(history.history['val_loss'], label='Validation')
axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print final metrics
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
best_val_acc = max(history.history['val_accuracy'])

print(f"Final Training Accuracy: {final_train_acc:.4f}")
print(f"Final Validation Accuracy: {final_val_acc:.4f}")
print(f"Best Validation Accuracy: {best_val_acc:.4f}")

## 🧪 Model Evaluation

In [ ]:
# Load best model
model = keras.models.load_model('best_model.h5')

# Evaluate on validation set
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Validation Loss: {val_loss:.4f}")

# Make predictions
y_pred = model.predict(X_val, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

cm = confusion_matrix(y_val, y_pred_classes)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=LABELS, yticklabels=LABELS)
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Classification report
print("\nClassification Report:")
print(classification_report(y_val, y_pred_classes, target_names=LABELS))

## 🎯 Test Predictions

In [ ]:
# Show sample predictions
num_samples = 16
indices = np.random.choice(len(X_val), num_samples, replace=False)

fig, axes = plt.subplots(4, 4, figsize=(16, 16))
fig.suptitle('Sample Predictions', fontsize=16, fontweight='bold')

for i, idx in enumerate(indices):
    img = X_val[idx]
    true_label = LABELS[y_val[idx]]
    
    # Predict
    pred = model.predict(img.reshape(1, IMG_SIZE, IMG_SIZE, 1), verbose=0)[0]
    pred_label = LABELS[np.argmax(pred)]
    confidence = np.max(pred) * 100
    
    # Plot
    row = i // 4
    col = i % 4
    axes[row, col].imshow(img.squeeze(), cmap='gray')
    
    color = 'green' if pred_label == true_label else 'red'
    axes[row, col].set_title(
        f"True: {true_label}\nPred: {pred_label} ({confidence:.1f}%)",
        fontsize=12, color=color, fontweight='bold'
    )
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

## 💾 Save Model

In [ ]:
# Save final model
model.save('asl_model_new.h5')
print("✓ Model saved as 'asl_model_new.h5'")

# Download model
files.download('asl_model_new.h5')
print("✓ Model downloaded!")

# Model info
import os
file_size = os.path.getsize('asl_model_new.h5') / (1024 * 1024)
print(f"\nModel size: {file_size:.2f} MB")
print(f"Input shape: (28, 28, 1)")
print(f"Output classes: {NUM_CLASSES}")
print(f"Validation accuracy: {val_acc:.4f}")

## 🧪 Test with Custom Image

In [ ]:
# Upload test image
print("Upload an ASL hand sign image to test...")
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"\nTesting: {filename}")
    
    # Preprocess
    img = preprocess_image(filename, use_crop=USE_HAND_CROP)
    img_input = img.reshape(1, IMG_SIZE, IMG_SIZE, 1)
    
    # Predict
    pred = model.predict(img_input, verbose=0)[0]
    top_3_idx = np.argsort(pred)[-3:][::-1]
    
    # Display
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Original image
    axes[0].imshow(img, cmap='gray')
    axes[0].set_title(f"Input Image (28x28)", fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    # Top 3 predictions
    labels_plot = [LABELS[i] for i in top_3_idx]
    probs_plot = [pred[i] * 100 for i in top_3_idx]
    
    axes[1].barh(labels_plot, probs_plot, color=['green', 'orange', 'red'])
    axes[1].set_xlabel('Confidence (%)', fontsize=12)
    axes[1].set_title('Top 3 Predictions', fontsize=12, fontweight='bold')
    axes[1].set_xlim(0, 100)
    
    for i, (label, prob) in enumerate(zip(labels_plot, probs_plot)):
        axes[1].text(prob + 2, i, f"{prob:.1f}%", va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nPrediction: {LABELS[top_3_idx[0]]}")
    print(f"Confidence: {pred[top_3_idx[0]]*100:.2f}%")
    print(f"\nTop 3:")
    for i in top_3_idx:
        print(f"  {LABELS[i]}: {pred[i]*100:.2f}%")

## 📝 Summary

In [ ]:
print("="*60)
print("TRAINING SUMMARY")
print("="*60)
print(f"Dataset: Kaggle ASL Alphabet")
print(f"Hand Cropping: {USE_HAND_CROP}")
print(f"Total images: {len(X)}")
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"")
print(f"Model architecture: CNN with BatchNorm + Dropout")
print(f"Total parameters: {model.count_params():,}")
print(f"")
print(f"Training epochs: {len(history.history['accuracy'])}")
print(f"Final training accuracy: {final_train_acc:.4f}")
print(f"Final validation accuracy: {final_val_acc:.4f}")
print(f"Best validation accuracy: {best_val_acc:.4f}")
print(f"")
print(f"Model saved: asl_model_new.h5 ({file_size:.2f} MB)")
print("="*60)
print("")
print("✓ Training completed successfully!")
print("✓ Model ready for deployment!")
print("")
print("NEXT STEPS:")
print("1. Download 'asl_model_new.h5'")
print("2. Replace old model in your project")
print("3. Test with real webcam")
print("="*60)